In [1]:
!pip install transformers -q

In [2]:
import numpy as np
import pandas as pd
import transformers
import torch
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch import nn
import warnings
warnings.filterwarnings("ignore")

In [3]:
from transformers import BertModel, BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
bert = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

#### Loading the dataset and pre-processing

In [6]:
data = pd.read_csv('/kaggle/input/forc-data/train.csv')

In [7]:
data

,abstract,author,doi,url,publication month,publication year,title,publisher,label,data_index
0,the production of b jets in association with a...,CMS Collaboration,10.1007/JHEP06(2012)126,NaN,NaN,NaN,Measurement of the Z/gamma* + b-jet cross sect...,JHEP 06 (2012) 126,Physics,44436
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",NaN,NaN,NaN,NaN,Modelling systemic price cojumps with Hawkes f...,NaN,Quantitative Finance,46308
2,large information sizes in samples and feature...,"David Banh, Alan Huang",NaN,NaN,NaN,NaN,Encoding large information structures in linea...,NaN,Machine Learning,52468
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",NaN,NaN,NaN,NaN,Hyperbolic polygonal billiards close to 1-dime...,NaN,Dynamics/Dynamical Systems,12449
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",10.1016/j.jag.2012.08.003,https://doi.org/10.1016/j.jag.2012.08.003,4,2013.0,analysis of aster data for mapping bauxite ric...,International Journal of Applied Earth Observa...,Sociology,43248
...,...,...,...,...,...,...,...,...,...,...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",10.1002/pmj.21366,NaN,10,2013.0,recruiting project managers: a comparative ana...,NaN,Sociology,52038
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),NaN,NaN,NaN,NaN,Optimal stopping for L\'evy processes and affi...,NaN,Statistics and Probability,20925
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",10.1103/PhysRevD.78.075006,NaN,NaN,NaN,Against Tachyophobia,"Phys.Rev.D78:075006,2008",Physics,16875
41538,NaN,"['Markus Mühling ', 'Ralph Ewerth', 'Joanna Ba...",10.1007/978-3-030-30760-8_9,NaN,9,2019.0,investigating correlations of inter-coder agre...,NaN,Computer Sciences,46341


In [8]:
data.columns

Index(['abstract', 'author', 'doi', 'url', 'publication month',
       'publication year', 'title', 'publisher', 'label', 'data_index'],
      dtype='object')

In [9]:
data.isnull().sum()

abstract              1120
author                 189
doi                  16977
url                  39593
publication month    34943
publication year     34936
title                    0
publisher            23942
label                    0
data_index               0
dtype: int64

In [10]:
data.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
data

,abstract,author,title,label
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics
41538,NaN,"['Markus Mühling ', 'Ralph Ewerth', 'Joanna Ba...",investigating correlations of inter-coder agre...,Computer Sciences


In [11]:
data.isnull().sum()

abstract    1120
author       189
title          0
label          0
dtype: int64

In [12]:
data = data.dropna()
data.isnull().sum()

abstract    0
author      0
title       0
label       0
dtype: int64

In [13]:
data.shape

(40332, 4)

In [14]:
data

,abstract,author,title,label
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41534,programs offered by academic institutions in h...,"['Alex Ferworn', 'Muthana Zouri']",an ontology-based approach for curriculum mapp...,Computer Engineering
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics


In [15]:
def NLP_cleaning(text):
    text_corpus = []
    for sent in tqdm(text, desc='Cleaning'):
        sent = re.sub('<[^>]*>', '', sent)
        sent = re.sub('[^a-zA-z0-9]', ' ', sent)
        sent = sent.lower()
        text_corpus.append(sent)

    return text_corpus

In [16]:
data['input'] = data['title']+": "+data['abstract']
data

,abstract,author,title,label,input
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics,Measurement of the Z/gamma* + b-jet cross sect...
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance,Modelling systemic price cojumps with Hawkes f...
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning,Encoding large information structures in linea...
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems,Hyperbolic polygonal billiards close to 1-dime...
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology,analysis of aster data for mapping bauxite ric...
...,...,...,...,...,...
41534,programs offered by academic institutions in h...,"['Alex Ferworn', 'Muthana Zouri']",an ontology-based approach for curriculum mapp...,Computer Engineering,an ontology-based approach for curriculum mapp...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology,recruiting project managers: a comparative ana...
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability,Optimal stopping for L\'evy processes and affi...
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics,Against Tachyophobia: we examine the possible ...


In [17]:
data['input'][0]

'Measurement of the Z/gamma* + b-jet cross section in pp collisions at 7\n  TeV: the production of b jets in association with a z/gamma* boson is studied using proton-proton collisions delivered by the lhc at a centre-of-mass energy of 7 tev and recorded by the cms detector. the inclusive cross section for z/gamma* + b-jet production is measured in a sample corresponding to an integrated luminosity of 2.2 inverse femtobarns. the z/gamma* + b-jet cross section with z/gamma* to ll (where ll = ee or mu mu) for events with the invariant mass 60 < m(ll) < 120 gev, at least one b jet at the hadron level with pt > 25 gev and abs(eta) < 2.1, and a separation between the leptons and the jets of delta r > 0.5 is found to be 5.84 +/- 0.08 (stat.) +/- 0.72 (syst.) +(0.25)/-(0.55) (theory) pb. the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia.

In [18]:
text = data.input.values.tolist()

In [19]:
text[0]

'Measurement of the Z/gamma* + b-jet cross section in pp collisions at 7\n  TeV: the production of b jets in association with a z/gamma* boson is studied using proton-proton collisions delivered by the lhc at a centre-of-mass energy of 7 tev and recorded by the cms detector. the inclusive cross section for z/gamma* + b-jet production is measured in a sample corresponding to an integrated luminosity of 2.2 inverse femtobarns. the z/gamma* + b-jet cross section with z/gamma* to ll (where ll = ee or mu mu) for events with the invariant mass 60 < m(ll) < 120 gev, at least one b jet at the hadron level with pt > 25 gev and abs(eta) < 2.1, and a separation between the leptons and the jets of delta r > 0.5 is found to be 5.84 +/- 0.08 (stat.) +/- 0.72 (syst.) +(0.25)/-(0.55) (theory) pb. the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia.

In [20]:
text_corpus = NLP_cleaning(text)

Cleaning: 100%|██████████| 40332/40332 [00:03<00:00, 11217.72it/s]


In [21]:
text_corpus[0]

'measurement of the z gamma    b jet cross section in pp collisions at 7   tev  the production of b jets in association with a z gamma  boson is studied using proton proton collisions delivered by the lhc at a centre of mass energy of 7 tev and recorded by the cms detector  the inclusive cross section for z gamma    b jet production is measured in a sample corresponding to an integrated luminosity of 2 2 inverse femtobarns  the z gamma    b jet cross section with z gamma  to ll  where ll   ee or mu mu  for events with the invariant mass 60  25 gev and abs eta   0 5 is found to be 5 84     0 08  stat       0 72  syst     0 25    0 55   theory  pb  the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia '

In [22]:
data['input'] = text_corpus

In [23]:
data['title'] = NLP_cleaning(data.title.values.tolist())
data['author'] = NLP_cleaning(data.author.values.tolist())

Cleaning: 100%|██████████| 40332/40332 [00:00<00:00, 82887.82it/s]


In [24]:
data

,abstract,author,title,label,input
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,Physics,measurement of the z gamma b jet cross sect...
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,Quantitative Finance,modelling systemic price cojumps with hawkes f...
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,Machine Learning,encoding large information structures in linea...
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems,hyperbolic polygonal billiards close to 1 dime...
4,Bauxite deposits of Jharkhand in India are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,Sociology,analysis of aster data for mapping bauxite ric...
...,...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,Computer Engineering,an ontology based approach for curriculum mapp...
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,Sociology,recruiting project managers a comparative ana...
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,Statistics and Probability,optimal stopping for l\ evy processes and affi...
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,Physics,against tachyophobia we examine the possible ...


In [25]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['label_number'] = label_encoder.fit_transform(data['label'])
data

,abstract,author,title,label,input,label_number
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,Physics,measurement of the z gamma b jet cross sect...,95
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,Quantitative Finance,modelling systemic price cojumps with hawkes f...,105
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,Machine Learning,encoding large information structures in linea...,67
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems,hyperbolic polygonal billiards close to 1 dime...,41
4,Bauxite deposits of Jharkhand in India are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,Sociology,analysis of aster data for mapping bauxite ric...,114
...,...,...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,Computer Engineering,an ontology based approach for curriculum mapp...,25
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,Sociology,recruiting project managers a comparative ana...,114
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,Statistics and Probability,optimal stopping for l\ evy processes and affi...,118
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,Physics,against tachyophobia we examine the possible ...,95


In [26]:
val_df = pd.read_csv('/kaggle/input/forc-data/val.csv')
val_df

,abstract,author,doi,url,publication month,publication year,title,publisher,label,data_index
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",10.1093/mnras/stv974,NaN,NaN,NaN,Observational Properties of Simulated Galaxies...,NaN,External Galaxies,22292
1,NaN,"['Yongzhuo Huang', 'Shengwu Huang', 'Yongxin Y...",10.1007/s13233-013-1029-2,NaN,10.0,2012.0,cell-penetrating peptide-modified plga nanopar...,Macromolecular Research,Medicinal Chemistry and Pharmaceutics,4465
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,NaN,NaN,NaN,NaN,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,NaN,Quantum Physics,16166
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",NaN,NaN,NaN,NaN,Suppression of magnetic phase transition at hi...,NaN,"Atomic, Molecular and Optical Physics",19452
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,NaN,NaN,NaN,NaN,Many neighborly spheres,NaN,Discrete Mathematics and Combinatorics,43731
...,...,...,...,...,...,...,...,...,...,...
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,10.1088/1361-6471/ac4e60,NaN,NaN,NaN,Energy correlation of bottom quarks from decay...,J. Phys. G: Nucl. Part. Phys. 49 045003 (2022),Physics,49058
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,NaN,NaN,NaN,NaN,Scattering processes in antiprotonic hydrogen ...,NaN,Nonlinear Sciences,26605
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",10.14778/1453856.1453873,NaN,8.0,2008.0,resisting structural re-identification in anon...,NaN,Earth Sciences,44741
8899,Predictive models of immune protection from CO...,"['Miles P. Davenport', 'James A. Triccas', 'St...",10.1038/s41591-021-01377-8,http://dx.doi.org/10.1038/s41591-021-01377-8,5.0,2021.0,neutralizing antibody levels are highly predic...,Nature Medicine,Microbiology,19564


In [27]:
val_df.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
val_df = val_df.dropna()
val_df['input'] = val_df['title']+": "+val_df['abstract']
val_df['input'] = NLP_cleaning(val_df.input.values.tolist())
val_df

Cleaning: 100%|██████████| 8648/8648 [00:00<00:00, 11257.85it/s]


,abstract,author,title,label,input
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",Observational Properties of Simulated Galaxies...,External Galaxies,observational properties of simulated galaxies...
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,Quantum Physics,su 3 yang mills hamiltonian in the flux tube ...
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",Suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics",suppression of magnetic phase transition at hi...
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,Many neighborly spheres,Discrete Mathematics and Combinatorics,many neighborly spheres the result of padrol ...
5,"using first-principles calculations, the elast...","Ming-Min Zhong, Xiao-Yu Kuang, Zhen-Hua Wang, ...",Physical properties of ReB2 under high pressur...,Materials Science and Engineering,physical properties of reb2 under high pressur...
...,...,...,...,...,...
8895,we present x-ray spectral analysis of seyfert ...,"Prantik Nandi, Sandip K. Chakrabarti, Santanu ...",Spectral properties of NGC 4151 and the Estima...,Physical Processes,spectral properties of ngc 4151 and the estima...
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,Energy correlation of bottom quarks from decay...,Physics,energy correlation of bottom quarks from decay...
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,Scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,scattering processes in antiprotonic hydrogen ...
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",resisting structural re-identification in anon...,Earth Sciences,resisting structural re identification in anon...


In [28]:
val_df['label_number'] = label_encoder.transform(val_df['label'])
val_df

,abstract,author,title,label,input,label_number
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",Observational Properties of Simulated Galaxies...,External Galaxies,observational properties of simulated galaxies...,51
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,Quantum Physics,su 3 yang mills hamiltonian in the flux tube ...,107
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",Suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics",suppression of magnetic phase transition at hi...,10
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,Many neighborly spheres,Discrete Mathematics and Combinatorics,many neighborly spheres the result of padrol ...,40
5,"using first-principles calculations, the elast...","Ming-Min Zhong, Xiao-Yu Kuang, Zhen-Hua Wang, ...",Physical properties of ReB2 under high pressur...,Materials Science and Engineering,physical properties of reb2 under high pressur...,68
...,...,...,...,...,...,...
8895,we present x-ray spectral analysis of seyfert ...,"Prantik Nandi, Sandip K. Chakrabarti, Santanu ...",Spectral properties of NGC 4151 and the Estima...,Physical Processes,spectral properties of ngc 4151 and the estima...,93
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,Energy correlation of bottom quarks from decay...,Physics,energy correlation of bottom quarks from decay...,95
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,Scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,scattering processes in antiprotonic hydrogen ...,84
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",resisting structural re-identification in anon...,Earth Sciences,resisting structural re identification in anon...,42


In [29]:
train_text, val_text, train_labels, val_labels = train_test_split(data.drop(['label','label_number'], axis = 1), data['label_number'],
                                                                    random_state=2018,
                                                                    test_size=0.15,
                                                                    stratify=data['label_number'])


test_text = val_df.drop(['label','label_number'], axis = 1)
test_labels = val_df['label_number']

In [30]:
test_text

,abstract,author,title,input
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",Observational Properties of Simulated Galaxies...,observational properties of simulated galaxies...
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,su 3 yang mills hamiltonian in the flux tube ...
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",Suppression of magnetic phase transition at hi...,suppression of magnetic phase transition at hi...
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,Many neighborly spheres,many neighborly spheres the result of padrol ...
5,"using first-principles calculations, the elast...","Ming-Min Zhong, Xiao-Yu Kuang, Zhen-Hua Wang, ...",Physical properties of ReB2 under high pressur...,physical properties of reb2 under high pressur...
...,...,...,...,...
8895,we present x-ray spectral analysis of seyfert ...,"Prantik Nandi, Sandip K. Chakrabarti, Santanu ...",Spectral properties of NGC 4151 and the Estima...,spectral properties of ngc 4151 and the estima...
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,Energy correlation of bottom quarks from decay...,energy correlation of bottom quarks from decay...
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,Scattering processes in antiprotonic hydrogen ...,scattering processes in antiprotonic hydrogen ...
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",resisting structural re-identification in anon...,resisting structural re identification in anon...


In [31]:
train_text

,abstract,author,title,input
1987,we study metabelian groups $g$ given by full r...,albert garreta leire legarreta alexei miasni...,metabelian groups full rank presentations ra...,metabelian groups full rank presentations ra...
9119,we present a generalized povm formalism for th...,george jaroszkiewicz,quantized detector network povms and the frans...,quantized detector network povms and the frans...
30407,we analyzed deep chandra observations of the g...,m revnivtsev 1 2 s sazonov 1 2 1 mpa...,on the contribution of point sources to the ga...,on the contribution of point sources to the ga...
16381,below a certain temperature tc (typically cryo...,maciej zgirski karri pekka riikonen vladimir...,size dependent breakdown of superconductivity ...,size dependent breakdown of superconductivity ...
7919,drees and rootz\'en [2010] have proven central...,jos\ e gregorio g\ omez,dependent lindeberg clt finite dimensional f...,dependent lindeberg clt finite dimensional f...
...,...,...,...,...
6980,the isometric path cover (partition) problem o...,paul manuel,on the isometric path partition problem,on the isometric path partition problem the i...
34637,we present the first formal semantics of finde...,andrei arusoaie,a formal semantics of findel in coq short paper,a formal semantics of findel in coq short pap...
24924,the current bottleneck of globally solving mix...,hongbo dong,relaxing nonconvex quadratic functions by mult...,relaxing nonconvex quadratic functions by mult...
27530,"in this paper, a hirota method is developed fo...",zai dong li qiu yan li xing hua hu zhong xi...,hirota method for the nonlinear schr\h o dinge...,hirota method for the nonlinear schr\h o dinge...


In [32]:
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
sent_id = tokenizer.batch_encode_plus(text, padding=True)
print(sent_id)

{'input_ids': [[101, 2023, 2003, 1037, 14324, 2944, 14924, 4818, 102, 0], [101, 2057, 2097, 2986, 1011, 8694, 1037, 14324, 2944, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [33]:
tokens_train = tokenizer.batch_encode_plus(
    train_text['input'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

tokens_val = tokenizer.batch_encode_plus(
    val_text['input'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text['input'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

In [34]:
len(pd.unique(data['label_number']))

123

In [35]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [36]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [37]:
for name, param in bert.named_parameters():
    print(f"{name}: {param.requires_grad}")

embeddings.word_embeddings.weight: True
embeddings.position_embeddings.weight: True
embeddings.token_type_embeddings.weight: True
embeddings.LayerNorm.weight: True
embeddings.LayerNorm.bias: True
encoder.layer.0.attention.self.query.weight: True
encoder.layer.0.attention.self.query.bias: True
encoder.layer.0.attention.self.key.weight: True
encoder.layer.0.attention.self.key.bias: True
encoder.layer.0.attention.self.value.weight: True
encoder.layer.0.attention.self.value.bias: True
encoder.layer.0.attention.output.dense.weight: True
encoder.layer.0.attention.output.dense.bias: True
encoder.layer.0.attention.output.LayerNorm.weight: True
encoder.layer.0.attention.output.LayerNorm.bias: True
encoder.layer.0.intermediate.dense.weight: True
encoder.layer.0.intermediate.dense.bias: True
encoder.layer.0.output.dense.weight: True
encoder.layer.0.output.dense.bias: True
encoder.layer.0.output.LayerNorm.weight: True
encoder.layer.0.output.LayerNorm.bias: True
encoder.layer.1.attention.self.query

In [38]:
name

'pooler.dense.bias'

In [39]:
# for param in bert.parameters():
#     param.requires_grad = False

for name, param in bert.named_parameters():
    if "pooler.dense.weight" in name:
        param.requires_grad = True
    elif "pooler.dense.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [40]:
for name, param in bert.named_parameters():
    print(f"{name}: {param.requires_grad}")

embeddings.word_embeddings.weight: False
embeddings.position_embeddings.weight: False
embeddings.token_type_embeddings.weight: False
embeddings.LayerNorm.weight: False
embeddings.LayerNorm.bias: False
encoder.layer.0.attention.self.query.weight: False
encoder.layer.0.attention.self.query.bias: False
encoder.layer.0.attention.self.key.weight: False
encoder.layer.0.attention.self.key.bias: False
encoder.layer.0.attention.self.value.weight: False
encoder.layer.0.attention.self.value.bias: False
encoder.layer.0.attention.output.dense.weight: False
encoder.layer.0.attention.output.dense.bias: False
encoder.layer.0.attention.output.LayerNorm.weight: False
encoder.layer.0.attention.output.LayerNorm.bias: False
encoder.layer.0.intermediate.dense.weight: False
encoder.layer.0.intermediate.dense.bias: False
encoder.layer.0.output.dense.weight: False
encoder.layer.0.output.dense.bias: False
encoder.layer.0.output.LayerNorm.weight: False
encoder.layer.0.output.LayerNorm.bias: False
encoder.layer.1

In [41]:
import torch
import torch.nn as nn
from transformers import BertModel

class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 123)  # Adjust the number of output neurons for 123 classes
        self.log_softmax = nn.LogSoftmax(dim=1)  # Use dim=1 for LogSoftmax in classification

    def forward(self, sent_id, mask):
        sent_id = torch.tensor(sent_id)

        outputs = self.bert(input_ids=sent_id, attention_mask=mask)
        last_hidden_state_cls = outputs[0][:, 0, :]

        x = self.fc1(last_hidden_state_cls)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.log_softmax(x)

        return x

In [42]:
model = BERT_Arch(bert)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [43]:
device

'cuda'

In [44]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 1e-4)

In [45]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_labels)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)
print("Class Weights:", class_weights)

Class Weights: [6.78139774e-01 5.90498829e-01 5.46500877e-01 5.16139717e+00
 5.33937638e-01 2.42361258e+00 7.14654993e-01 6.33444198e+00
 1.25944621e-01 8.70985772e+00 3.47959360e-01 2.78715447e+01
 3.71620596e-01 2.01967715e+00 5.57430894e+01 8.44592264e+00
 4.64525745e+01 1.67900872e+00 3.09683830e+01 1.39357724e+01
 1.82166959e+00 7.53284992e+00 5.25878202e+00 4.53195849e-01
 5.80657182e+00 2.28455285e+00 3.81801982e+00 7.65701778e-01
 2.80397834e-01 2.36199531e+00 2.51094997e+00 3.74114694e-01
 3.98164925e+00 5.56318258e-01 3.95341060e-01 8.42040626e-01
 1.39357724e+01 1.38664402e+00 6.32007817e-01 6.05903146e+00
 4.15373245e-01 2.70335060e-01 4.28792996e+00 1.60181291e+00
 1.26688840e+01 1.63950263e+01 2.46650838e+00 8.99082088e+00
 1.39357724e+02 1.85810298e+00 4.09875658e+00 4.22936946e-01
 1.18602318e+00 1.21180629e+01 3.51469669e-01 5.80657182e+00
 1.21180629e+01 8.44592264e+00 1.21180629e+01 2.40271937e+00
 8.99082088e+00 1.05574033e+00 3.76642496e+00 9.38435849e-01
 2.815307

In [46]:
weights= torch.tensor(class_weights,dtype=torch.float)
weights = weights.to(device)
cross_entropy  = nn.NLLLoss(weight=weights)

epochs = 1

In [47]:
def train():

    model.train()
    total_loss, total_accuracy = 0, 0
    total_preds=[]

    for step,batch in enumerate(train_dataloader):

        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch
        sent_id = torch.tensor(sent_id)

        model.zero_grad()
        preds = model(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss = total_loss + loss.item()
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        preds=preds.detach().cpu().numpy()
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [48]:
import time
def evaluate():

    print("\nEvaluating...")
    model.eval()
    total_loss, total_accuracy = 0, 0
    total_preds = []

    for step,batch in enumerate(val_dataloader):

        if step % 50 == 0 and not step == 0:
#             elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch
    sent_id = torch.tensor(sent_id)

    with torch.no_grad():

        preds = model(sent_id, mask)
        loss = cross_entropy(preds,labels)
        total_loss = total_loss + loss.item()
        preds = preds.detach().cpu().numpy()
        total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [49]:
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, _ = train()
    valid_loss, _ = evaluate()
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 1
  Batch    50  of  2,143.
  Batch   100  of  2,143.
  Batch   150  of  2,143.
  Batch   200  of  2,143.
  Batch   250  of  2,143.
  Batch   300  of  2,143.
  Batch   350  of  2,143.
  Batch   400  of  2,143.
  Batch   450  of  2,143.
  Batch   500  of  2,143.
  Batch   550  of  2,143.
  Batch   600  of  2,143.
  Batch   650  of  2,143.
  Batch   700  of  2,143.
  Batch   750  of  2,143.
  Batch   800  of  2,143.
  Batch   850  of  2,143.
  Batch   900  of  2,143.
  Batch   950  of  2,143.
  Batch 1,000  of  2,143.
  Batch 1,050  of  2,143.
  Batch 1,100  of  2,143.
  Batch 1,150  of  2,143.
  Batch 1,200  of  2,143.
  Batch 1,250  of  2,143.
  Batch 1,300  of  2,143.
  Batch 1,350  of  2,143.
  Batch 1,400  of  2,143.
  Batch 1,450  of  2,143.
  Batch 1,500  of  2,143.
  Batch 1,550  of  2,143.
  Batch 1,600  of  2,143.
  Batch 1,650  of  2,143.
  Batch 1,700  of  2,143.
  Batch 1,750  of  2,143.
  Batch 1,800  of  2,143.
  Batch 1,850  of  2,143.
  Batch 1,900  of  2,143

In [50]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint2.pth')

In [51]:
input_text = input('Give an input text : ')
tokens = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

Give an input text :  hell  bro


In [54]:
def Predict(text):
    encoded_review = tokenizer.encode_plus(
      text,
      max_length=512,
      add_special_tokens=True,
      return_token_type_ids=False,
      truncation = True,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    extracted_value = prediction.item()
    return extracted_value

In [55]:
y_pred = []
for text in test_text['input']:
    val = Predict(text)
    y_pred.append(val)

In [56]:
len(y_pred)

8648

In [57]:
test_labels

0        51
2       107
3        10
4        40
5        68
       ... 
8895     93
8896     95
8897     84
8898     42
8899     77
Name: label_number, Length: 8648, dtype: int64

In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, classification_report
import numpy as np

true_labels = test_labels
predicted_labels = y_pred

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.3323


In [59]:
print(classification_report(true_labels, y_pred))

              precision    recall  f1-score   support

           0       0.33      0.14      0.20        99
           1       0.19      0.89      0.31       126
           2       0.25      0.23      0.24       130
           3       0.00      0.00      0.00        12
           4       0.00      0.00      0.00       118
           5       0.15      0.13      0.14        30
           6       0.19      0.16      0.18       104
           7       1.00      0.11      0.20         9
           8       0.54      0.39      0.45       557
           9       0.00      0.00      0.00        11
          10       0.00      0.00      0.00       225
          11       0.00      0.00      0.00         3
          12       0.57      0.74      0.64       189
          13       0.05      0.57      0.09        42
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         9
          16       0.00      0.00      0.00         2
          17       0.18    